In [1]:
import os

In [2]:
list_menu = list(os.walk('./menu'))[0][2]
if '.DS_Store' in list_menu:
    list_menu.remove('.DS_Store')
if '0.null.txt' in list_menu:
    list_menu.remove('0.null.txt')
    
list_menu.sort()
    
len(list_menu)

32

In [3]:
for menu in list_menu:

    param_market = 'UK'

    from gevent import monkey
    monkey.patch_all(thread=False)
    from gevent.queue import Queue
    import gevent

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct = pd.DataFrame()
    output_error = pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel('./menu/' + menu,
                           header=0,
                           dtype=str).fillna('')

    print('总数量：' + str(len(input_)))
    print()

    # = = = = = = = = = = = = = = =

    import requests

    import sys
    sys.path.append('../../00.Tools')
    import UA
    import Proxy

    # = = = = = = = = = = = = = = =

    import json

    # = = = = = = = = = = = = = = =

    import time
    from datetime import datetime

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'
            for b in range(666):
                try:
                    if param_market == 'US':
                        url = 'https://www.ebay.com/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                    elif param_market == 'DE':
                        url = 'https://www.ebay.de/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=DE&_stpos=10115&orig_cvip=true'
                    elif param_market == 'AU':
                        url = 'https://www.ebay.com.au/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=AU&_stpos=2019&orig_cvip=true'
                    elif param_market == 'UK':
                        url = 'https://www.ebay.co.uk/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=GB&_stpos=BS81QU&orig_cvip=true'

                    resp = requests.get(url,
                                        #headers=UA.get_User_Agent_Requests(),
                                        proxies=Proxy.get_Proxy_Requests()).text

                    # = = = = = = = = = = = = = = =

                    json_data = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script></body></html>')[0] + ']}]}')

                    # = = = = = = = = = = = = = = =

                    market_id = json_data['o']['g']['marketplaceId'].strip()

                    # = = = = = = = = = = = = = = =

                    country = json_data['o']['g']['country'].strip()

                    # = = = = = = = = = = = = = = =

                    language = json_data['o']['g']['language'].strip()

                    # = = = = = = = = = = = = = = =

                    locale = json_data['o']['g']['locale'].strip()

                    # = = = = = = = = = = = = = = =

                    item_id = json_data['o']['g']['itemId'].strip()

                    # = = = = = = = = = = = = = = =

                    if param_market == 'US':
                        if market_id != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US' or item_id != input_.loc[a, 'Item_Number']:
                            continue
                    if param_market == 'AU':
                        if market_id != 'EBAY-AU' or country != 'AU' or language != 'en' or locale != 'en-AU' or item_id != input_.loc[a, 'Item_Number']:
                            continue
                    elif param_market == 'UK':
                        if market_id != 'EBAY-GB' or country != 'GB' or language != 'en' or locale != 'en-GB' or item_id != input_.loc[a, 'Item_Number']:
                            continue

                    # = = = = = = = = = = = = = = =

                    specification = json_data['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                    # = = = = = = = = = = = = = = =

                    if param_market in ['US', 'AU', 'UK']:
                        if 'VEHICLE_SELECTION' in json_data['o']['w'][0][2]['model']['modules']:
                            category_id = json_data['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                            # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                            vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                        else:
                            category_id = ''
                            vehicle_page = ''
                    elif param_market == 'DE':
                        if 'MOTORS_TAB' in json_data['o']['w'][0][2]['model']['modules']:
                            category_id = json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleHsnTsnLookupTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                            # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                            vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('von')[1].strip()
                        else:
                            category_id = ''
                            vehicle_page = ''

                    # = = = = = = = = = = = = = = =

                    title = json_data['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                    # = = = = = = = = = = = = = = =

                    if 'subTitle' in json_data['o']['w'][0][2]['model']['modules']['TITLE']:
                        subtitle = json_data['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                    else:
                        subtitle = ''

                    # = = = = = = = = = = = = = = =

                    if 'binModel' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                        price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                    elif 'bidPrice' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                        price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                    # = = = = = = = = = = = = = = =

                    if 'QUANTITY' in json_data['o']['w'][0][2]['model']['modules']:
                        available = ''
                        sold = ''
                        list_text = json_data['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                        for text in list_text:
                            if param_market in ['US', 'AU', 'UK']:
                                if 'available' in text['text']:
                                    available = text['text'].strip()
                                elif 'sold' in text['text']:
                                    sold = text['text'].strip()
                            elif param_market == 'DE':
                                if 'verfügbar' in text['text']:
                                    available = text['text'].strip()
                                elif 'verkauft' in text['text']:
                                    sold = text['text'].strip()
                    else:
                        available = ''
                        sold = ''

                    # = = = = = = = = = = = = = = =

                    if 'mediaList' not in json_data['o']['w'][0][2]['model']['modules']['PICTURE']:
                        src = ''
                    elif 'largeImg' in json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                        src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                    else:
                        src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                    # = = = = = = = = = = = = = = =

                    if 'action' in json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]:
                        description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()
                    else:
                        description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL']

                    # = = = = = = = = = = = = = = =

                    df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                             'Item_Number': input_.loc[a, 'Item_Number'],
                                             'Title': title,
                                             'Subtitle': subtitle,
                                             'Price': price,
                                             'Available': available,
                                             'Sold': sold,
                                             'Vehicle_1': '',
                                             'Vehicle_2': '',
                                             'Pic': '',
                                             'Url': url,
                                             'Description_Url': description_url,
                                             'Src': src,
                                             'Vehicle_Page': vehicle_page,
                                             'Category_Id': category_id,
                                             'Market_Id': market_id,
                                             'Country': country,
                                             'Language': language,
                                             'Locale': locale}])

                    dict_name = {}
                    if param_market in ['US', 'AU', 'UK']:
                        condition = 'Condition'
                    elif param_market == 'DE':
                        condition = 'Artikelzustand'

                    for name_value in specification:
                        name = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                        if name == condition:
                            df_temp[condition] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in specification[name_value]['values'][0] else specification[name_value]['values'][0]['textSpans'][0]['text'].strip()
                        else:
                            name = name.lower()
                            if name in dict_name:
                                dict_name[name] += 1
                            else:
                                dict_name[name] = 1
                            df_temp[name if dict_name[name] == 1 else name + '-' + str(dict_name[name])] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in specification[name_value]['values'][0] else specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                    # = = = = = = = = = = = = = = =

                    crawler_status = 'ok'

                    break
                except:
                    time.sleep(0.3)
                    continue

            # = = = = = = = = = = = = = = =

            if crawler_status == 'error':
                df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                         'Item_Number': input_.loc[a, 'Item_Number']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            print('[' + crawler_status + '] - ' + input_.loc[a, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(20):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    output_correct = output_correct.drop_duplicates(ignore_index=True)
    output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
    output_correct.to_excel('./item/' + menu.split('.xlsx')[0] + '-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
    if len(output_error) != 0:
        output_error = output_error.drop_duplicates(ignore_index=True)
        output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
        output_error.to_excel('./item/' + menu.split('.xlsx')[0] + '_error.xlsx', index=False)
        print('爬虫存在error')
        print()
print('Done ~')

总数量：171

[ok] - 171918669827
[尝试次数：1] - [剩余数量：151] - [当前时间：06:11:45]

[ok] - 164071507370
[尝试次数：1] - [剩余数量：150] - [当前时间：06:11:45]

[ok] - 164071496994
[尝试次数：1] - [剩余数量：149] - [当前时间：06:11:45]

[ok] - 174184344004
[尝试次数：1] - [剩余数量：148] - [当前时间：06:11:45]

[ok] - 153824676751
[尝试次数：1] - [剩余数量：147] - [当前时间：06:11:45]

[ok] - 160597146385
[尝试次数：1] - [剩余数量：146] - [当前时间：06:11:46]

[ok] - 164971387118
[尝试次数：1] - [剩余数量：145] - [当前时间：06:11:46]

[ok] - 153422739554
[尝试次数：1] - [剩余数量：144] - [当前时间：06:11:46]

[ok] - 174184340795
[尝试次数：1] - [剩余数量：143] - [当前时间：06:11:46]

[ok] - 173557246735
[尝试次数：1] - [剩余数量：142] - [当前时间：06:11:46]

[ok] - 162889524266
[尝试次数：1] - [剩余数量：141] - [当前时间：06:11:46]

[ok] - 151801834171
[尝试次数：1] - [剩余数量：140] - [当前时间：06:11:46]

[ok] - 153422735951
[尝试次数：1] - [剩余数量：139] - [当前时间：06:11:46]

[ok] - 174184350035
[尝试次数：1] - [剩余数量：138] - [当前时间：06:11:47]

[ok] - 161885618492
[尝试次数：1] - [剩余数量：137] - [当前时间：06:11:47]

[ok] - 163786832387
[尝试次数：1] - [剩余数量：136] - [当前时间：06:11:47]

[ok] - 15061341

[ok] - 171905260336
[尝试次数：1] - [剩余数量：14] - [当前时间：06:12:08]

[ok] - 153424904445
[尝试次数：1] - [剩余数量：13] - [当前时间：06:12:09]

[ok] - 162890949417
[尝试次数：1] - [剩余数量：12] - [当前时间：06:12:09]

[ok] - 153570911308
[尝试次数：1] - [剩余数量：11] - [当前时间：06:12:10]

[ok] - 173867971164
[尝试次数：1] - [剩余数量：10] - [当前时间：06:12:10]

[ok] - 150612968178
[尝试次数：1] - [剩余数量：9] - [当前时间：06:12:10]

[ok] - 161814267009
[尝试次数：1] - [剩余数量：8] - [当前时间：06:12:10]

[ok] - 173557251839
[尝试次数：1] - [剩余数量：7] - [当前时间：06:12:10]

[ok] - 163268725886
[尝试次数：1] - [剩余数量：6] - [当前时间：06:12:10]

[ok] - 172763671861
[尝试次数：1] - [剩余数量：5] - [当前时间：06:12:10]

[ok] - 151801834532
[尝试次数：1] - [剩余数量：4] - [当前时间：06:12:10]

[ok] - 174511726957
[尝试次数：1] - [剩余数量：3] - [当前时间：06:12:11]

[ok] - 152897583278
[尝试次数：1] - [剩余数量：2] - [当前时间：06:12:11]

[ok] - 171905290794
[尝试次数：1] - [剩余数量：1] - [当前时间：06:12:11]

[ok] - 153687935680
[尝试次数：1] - [剩余数量：0] - [当前时间：06:12:11]

[ok] - 161804204474
[尝试次数：1] - [剩余数量：0] - [当前时间：06:12:11]

[ok] - 173843857295
[尝试次数：1] - [剩余数量：0] - [当前时间：06:

[ok] - 171184125021
[尝试次数：1] - [剩余数量：548] - [当前时间：06:12:43]

[ok] - 170774127917
[尝试次数：1] - [剩余数量：547] - [当前时间：06:12:43]

[ok] - 171790526598
[尝试次数：1] - [剩余数量：546] - [当前时间：06:12:43]

[ok] - 161141015098
[尝试次数：1] - [剩余数量：545] - [当前时间：06:12:44]

[ok] - 151682338543
[尝试次数：1] - [剩余数量：544] - [当前时间：06:12:44]

[ok] - 151155830196
[尝试次数：1] - [剩余数量：543] - [当前时间：06:12:44]

[ok] - 171426707691
[尝试次数：1] - [剩余数量：542] - [当前时间：06:12:44]

[ok] - 171184119215
[尝试次数：1] - [剩余数量：541] - [当前时间：06:12:44]

[ok] - 161162118329
[尝试次数：1] - [剩余数量：540] - [当前时间：06:12:44]

[ok] - 151176962075
[尝试次数：1] - [剩余数量：539] - [当前时间：06:12:44]

[ok] - 160740514481
[尝试次数：1] - [剩余数量：538] - [当前时间：06:12:44]

[ok] - 150832824438
[尝试次数：1] - [剩余数量：537] - [当前时间：06:12:44]

[ok] - 160748932828
[尝试次数：1] - [剩余数量：536] - [当前时间：06:12:45]

[ok] - 161398782163
[尝试次数：1] - [剩余数量：535] - [当前时间：06:12:45]

[ok] - 161798449291
[尝试次数：1] - [剩余数量：534] - [当前时间：06:12:45]

[ok] - 171789509623
[尝试次数：1] - [剩余数量：533] - [当前时间：06:12:45]

[ok] - 150601278842
[尝试次

[ok] - 173446115198
[尝试次数：1] - [剩余数量：412] - [当前时间：06:13:07]

[ok] - 150566861580
[尝试次数：2] - [剩余数量：411] - [当前时间：06:13:07]

[ok] - 170693678826
[尝试次数：1] - [剩余数量：410] - [当前时间：06:13:07]

[ok] - 151423509850
[尝试次数：1] - [剩余数量：409] - [当前时间：06:13:08]

[ok] - 150555004093
[尝试次数：1] - [剩余数量：408] - [当前时间：06:13:08]

[ok] - 171163434996
[尝试次数：2] - [剩余数量：407] - [当前时间：06:13:08]

[ok] - 151682332028
[尝试次数：1] - [剩余数量：406] - [当前时间：06:13:08]

[ok] - 160570898546
[尝试次数：1] - [剩余数量：405] - [当前时间：06:13:08]

[ok] - 151682335873
[尝试次数：1] - [剩余数量：404] - [当前时间：06:13:09]

[ok] - 170910257635
[尝试次数：1] - [剩余数量：403] - [当前时间：06:13:09]

[ok] - 161706079877
[尝试次数：1] - [剩余数量：402] - [当前时间：06:13:09]

[ok] - 160747980450
[尝试次数：1] - [剩余数量：401] - [当前时间：06:13:09]

[ok] - 171790522148
[尝试次数：1] - [剩余数量：400] - [当前时间：06:13:09]

[ok] - 151202296370
[尝试次数：1] - [剩余数量：399] - [当前时间：06:13:09]

[ok] - 170915342890
[尝试次数：1] - [剩余数量：398] - [当前时间：06:13:10]

[ok] - 170890505571
[尝试次数：1] - [剩余数量：397] - [当前时间：06:13:10]

[ok] - 170890434546
[尝试次

[ok] - 161398918265
[尝试次数：2] - [剩余数量：277] - [当前时间：06:13:33]

[ok] - 150944685345
[尝试次数：1] - [剩余数量：276] - [当前时间：06:13:33]

[ok] - 161347657862
[尝试次数：1] - [剩余数量：275] - [当前时间：06:13:33]

[ok] - 171115397871
[尝试次数：1] - [剩余数量：274] - [当前时间：06:13:33]

[ok] - 161285698858
[尝试次数：1] - [剩余数量：273] - [当前时间：06:13:34]

[ok] - 151337384168
[尝试次数：1] - [剩余数量：272] - [当前时间：06:13:34]

[ok] - 150603905449
[尝试次数：1] - [剩余数量：271] - [当前时间：06:13:34]

[ok] - 171367199795
[尝试次数：1] - [剩余数量：270] - [当前时间：06:13:34]

[ok] - 151793292977
[尝试次数：1] - [剩余数量：269] - [当前时间：06:13:35]

[ok] - 151110509324
[尝试次数：1] - [剩余数量：268] - [当前时间：06:13:35]

[ok] - 170900439484
[尝试次数：1] - [剩余数量：267] - [当前时间：06:13:36]

[ok] - 171115323311
[尝试次数：1] - [剩余数量：266] - [当前时间：06:13:36]

[ok] - 151110509831
[尝试次数：1] - [剩余数量：265] - [当前时间：06:13:36]

[ok] - 150551792227
[尝试次数：1] - [剩余数量：264] - [当前时间：06:13:36]

[ok] - 151110509604
[尝试次数：1] - [剩余数量：263] - [当前时间：06:13:36]

[ok] - 171367198755
[尝试次数：1] - [剩余数量：262] - [当前时间：06:13:36]

[ok] - 171183414827
[尝试次

...

[ok] - 160898645047
[尝试次数：1] - [剩余数量：187] - [当前时间：07:31:14]

[ok] - 160898643429
[尝试次数：1] - [剩余数量：186] - [当前时间：07:31:15]

[ok] - 150920744977
[尝试次数：1] - [剩余数量：185] - [当前时间：07:31:15]

[ok] - 160898640436
[尝试次数：1] - [剩余数量：184] - [当前时间：07:31:15]

[ok] - 170922391712
[尝试次数：1] - [剩余数量：183] - [当前时间：07:31:15]

[ok] - 160898645691
[尝试次数：1] - [剩余数量：182] - [当前时间：07:31:15]

[ok] - 150920751347
[尝试次数：1] - [剩余数量：181] - [当前时间：07:31:15]

[ok] - 150920752625
[尝试次数：1] - [剩余数量：180] - [当前时间：07:31:15]

[ok] - 160898644130
[尝试次数：1] - [剩余数量：179] - [当前时间：07:31:15]

[ok] - 170922389381
[尝试次数：1] - [剩余数量：178] - [当前时间：07:31:15]

[ok] - 160898644478
[尝试次数：1] - [剩余数量：177] - [当前时间：07:31:16]

[ok] - 150920755977
[尝试次数：1] - [剩余数量：176] - [当前时间：07:31:17]

[ok] - 170922389124
[尝试次数：1] - [剩余数量：175] - [当前时间：07:31:17]

[ok] - 170922392648
[尝试次数：1] - [剩余数量：174] - [当前时间：07:31:17]

[ok] - 150920754892
[尝试次数：1] - [剩余数量：173] - [当前时间：07:31:17]

[ok] - 170900882366
[尝试次数：1] - [剩余数量：172] - [当前时间：07:31:18]

[ok] - 150918375649
[尝试次

[ok] - 170791006491
[尝试次数：1] - [剩余数量：50] - [当前时间：07:31:39]

[ok] - 150923498883
[尝试次数：1] - [剩余数量：49] - [当前时间：07:31:39]

[ok] - 170924232956
[尝试次数：1] - [剩余数量：48] - [当前时间：07:31:39]

[ok] - 160900848615
[尝试次数：1] - [剩余数量：47] - [当前时间：07:31:40]

[ok] - 160900673044
[尝试次数：1] - [剩余数量：46] - [当前时间：07:31:40]

[ok] - 170791005372
[尝试次数：1] - [剩余数量：45] - [当前时间：07:31:40]

[ok] - 160701550193
[尝试次数：1] - [剩余数量：44] - [当前时间：07:31:40]

[ok] - 170924233867
[尝试次数：1] - [剩余数量：43] - [当前时间：07:31:40]

[ok] - 170924233265
[尝试次数：1] - [剩余数量：42] - [当前时间：07:31:40]

[ok] - 150923501655
[尝试次数：1] - [剩余数量：41] - [当前时间：07:31:40]

[ok] - 170791006261
[尝试次数：1] - [剩余数量：40] - [当前时间：07:31:40]

[ok] - 160628807859
[尝试次数：1] - [剩余数量：39] - [当前时间：07:31:41]

[ok] - 170791006601
[尝试次数：1] - [剩余数量：38] - [当前时间：07:31:41]

[ok] - 170674125291
[尝试次数：1] - [剩余数量：37] - [当前时间：07:31:41]

[ok] - 170675798269
[尝试次数：1] - [剩余数量：36] - [当前时间：07:31:41]

[ok] - 160900673377
[尝试次数：2] - [剩余数量：35] - [当前时间：07:31:41]

[ok] - 170676314702
[尝试次数：1] - [剩余数量：34]

[ok] - 172968354016
[尝试次数：1] - [剩余数量：281] - [当前时间：07:32:08]

[ok] - 152777767242
[尝试次数：1] - [剩余数量：280] - [当前时间：07:32:08]

[ok] - 150590449197
[尝试次数：1] - [剩余数量：279] - [当前时间：07:32:09]

[ok] - 150603135913
[尝试次数：1] - [剩余数量：278] - [当前时间：07:32:09]

[ok] - 151008282021
[尝试次数：1] - [剩余数量：277] - [当前时间：07:32:09]

[ok] - 172968358693
[尝试次数：1] - [剩余数量：276] - [当前时间：07:32:09]

[ok] - 150590464404
[尝试次数：1] - [剩余数量：275] - [当前时间：07:32:10]

[ok] - 171797518089
[尝试次数：1] - [剩余数量：274] - [当前时间：07:32:10]

[ok] - 151227259375
[尝试次数：2] - [剩余数量：273] - [当前时间：07:32:10]

[ok] - 150563938366
[尝试次数：1] - [剩余数量：272] - [当前时间：07:32:10]

[ok] - 150652600641
[尝试次数：1] - [剩余数量：271] - [当前时间：07:32:10]

[ok] - 150563939359
[尝试次数：1] - [剩余数量：270] - [当前时间：07:32:10]

[ok] - 150707343732
[尝试次数：1] - [剩余数量：269] - [当前时间：07:32:10]

[ok] - 161711931404
[尝试次数：1] - [剩余数量：268] - [当前时间：07:32:11]

[ok] - 160641032349
[尝试次数：1] - [剩余数量：267] - [当前时间：07:32:11]

[ok] - 170727696598
[尝试次数：1] - [剩余数量：266] - [当前时间：07:32:11]

[ok] - 170900457268
[尝试次

[ok] - 150640024578
[尝试次数：1] - [剩余数量：143] - [当前时间：07:32:32]

[ok] - 160568950301
[尝试次数：1] - [剩余数量：142] - [当前时间：07:32:32]

[ok] - 150652691010
[尝试次数：1] - [剩余数量：141] - [当前时间：07:32:33]

[ok] - 160641130601
[尝试次数：1] - [剩余数量：140] - [当前时间：07:32:33]

[ok] - 160641130637
[尝试次数：1] - [剩余数量：139] - [当前时间：07:32:33]

[ok] - 160601132590
[尝试次数：1] - [剩余数量：138] - [当前时间：07:32:34]

[ok] - 170685627542
[尝试次数：1] - [剩余数量：137] - [当前时间：07:32:34]

[ok] - 150652691106
[尝试次数：1] - [剩余数量：136] - [当前时间：07:32:34]

[ok] - 160871954920
[尝试次数：1] - [剩余数量：135] - [当前时间：07:32:34]

[ok] - 150652634392
[尝试次数：1] - [剩余数量：134] - [当前时间：07:32:34]

[ok] - 170700045764
[尝试次数：1] - [剩余数量：133] - [当前时间：07:32:34]

[ok] - 150624225207
[尝试次数：1] - [剩余数量：132] - [当前时间：07:32:34]

[ok] - 170685627469
[尝试次数：1] - [剩余数量：131] - [当前时间：07:32:34]

[ok] - 160689286787
[尝试次数：1] - [剩余数量：130] - [当前时间：07:32:34]

[ok] - 150652634352
[尝试次数：1] - [剩余数量：129] - [当前时间：07:32:34]

[ok] - 150887492532
[尝试次数：1] - [剩余数量：128] - [当前时间：07:32:35]

[ok] - 160986296511
[尝试次

[ok] - 170697313969
[尝试次数：1] - [剩余数量：6] - [当前时间：07:32:56]

[ok] - 173030641404
[尝试次数：1] - [剩余数量：5] - [当前时间：07:32:56]

[ok] - 173898232372
[尝试次数：1] - [剩余数量：4] - [当前时间：07:32:56]

[ok] - 150667836591
[尝试次数：1] - [剩余数量：3] - [当前时间：07:32:56]

[ok] - 150663298208
[尝试次数：1] - [剩余数量：2] - [当前时间：07:32:56]

[ok] - 150624220309
[尝试次数：1] - [剩余数量：1] - [当前时间：07:32:57]

[ok] - 162014859646
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:57]

[ok] - 161044876514
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:57]

[ok] - 150661167843
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:57]

[ok] - 151012981774
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:58]

[ok] - 151062898364
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:58]

[ok] - 162801403719
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:58]

[ok] - 150707344013
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:58]

[ok] - 153488485789
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:58]

[ok] - 150661167731
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:59]

[ok] - 171057394754
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:59]

[ok] - 150662467042
[尝试次数：1] - [剩余数量：0] - [当前时间：07:32:59